### Creating World

In [15]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.
"""

In [16]:
world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""


In [ ]:
from huggingface_hub import login
login()

In [ ]:
import transformers
import torch

model_id = 'meta-llama/Llama-3.2-3B-Instruct'

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map= 0,
)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": world_prompt},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)




In [ ]:
world_output = outputs[0]["generated_text"][-1]
world_output = world_output['content']
print(world_output)

In [20]:
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('World Description:', '').strip()
}

In [ ]:
print(world)

### Generating Kingdoms

In [22]:
kingdom_prompt = f"""
Create 2 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""


In [ ]:
import torch
import gc

# After each generation step
torch.cuda.empty_cache()
gc.collect()


In [ ]:

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": kingdom_prompt},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)

In [ ]:
kingdoms = {}
kingdoms_output = outputs[0]["generated_text"][-1]
kingdoms_output = kingdoms_output['content']

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0] \
    .split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  kingdom_description = output.strip().split('\n')[1] \
    .split('Description: ')[1].strip()
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name']
  }
  kingdoms[kingdom_name] = kingdom
world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: \
{kingdom["description"]}')

### Generating Towns

In [34]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 2 different towns for a fantasy kingdom abd world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""

In [ ]:
import torch
import gc

# After each generation step
torch.cuda.empty_cache()
gc.collect()


In [36]:
def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": get_town_prompt(world,kingdom)},
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    
    towns_output = outputs[0]["generated_text"][-1]
    towns_output = towns_output['content']
    
    towns = {}
    for output in towns_output.split('\n\n')[:2]:
        town_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- {town_name} created')
        
        town_description = output.strip().split('\n')[1]\
        .split('Description: ')[1].strip()
        
        town = {
          "name": town_name,
          "description": town_description,
          "world": world['name'],
          "kingdom": kingdom['name']
        }
        towns[town_name] = town
    kingdom["towns"] = towns

In [ ]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

town = list(kingdom['towns'].values())[0]
print(f'\nTown 1 Description: \
{town["description"]}')

### Generating Non-Player Characters (NPC's)

In [64]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create exactly 2 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""

In [ ]:
import torch
import gc

# After each generation step
torch.cuda.empty_cache()
gc.collect()


In [66]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": get_npc_prompt(world,kingdom,town)},
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=256,
        temperature = 1,
    )

    npcs_output = outputs[0]["generated_text"][-1]
    npcs_output = npcs_output['content']
    
    
    npcs = {}
    for output in npcs_output.split('\n\n')[:2]:
        npc_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- "{npc_name}" created')
        
        npc_description = output.strip().split('\n')[1\
        ].split('Description: ')[1].strip()
        
        npc = {
        "name": npc_name,
        "description": npc_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[npc_name] = npc
    town["npcs"] = npcs

In [ ]:
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom
    break

In [ ]:
npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')

### Saving World

In [69]:
import json

def save_world(world, filename):
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)

#save_world(world, '../shared_data/Kyropeia.json')
save_world(world, r"C:\Users\arnav\OneDrive\Desktop\ai_powered_dm\shared_data\Kyrethys.json") #save to your version